# Distributions

In [37]:
import pandas as pd

from pyrlap.pyrlap2.core import Multinomial as Pr
from pyrlap.pyrlap2.core import AssignmentMap as Dict

def toDF(p):
    df = pd.DataFrame(p.support)
    df['prob'] = p.probs
    df['logit'] = p.logits
    return df

def assign(dist, variable, value):
    """Creates a new variable with a value for every element of support of dist"""
    return Pr(
        support=[{variable: value, **s} for s in dist.support],
        logits=dist.logits
    )

def marginalize(dist, projection):
    if isinstance(projection, Hashable):
        proj = Dict()
        for s, logit in dist.items():
            s = copy.deepcopy(s)
            del s[projection]
            proj[s] = proj.get(s, 0)
            proj[s] += np.exp(logit)
        support, probs = list(zip(*proj.items()))
        return Pr(support=support, probs=probs)
    else:
        raise NotImplementedError

# Conjunction / Products
The conjunction of two distributions $p_{0}(A, B)$ and $p_1(B, C)$ is $p_2(A, B, C)$ where:
$$
p_2(A, B, C) \propto p_0(A, B)p_1(B, C)
$$

In [11]:
# Conjunction (independent variables)
pA = Pr([{'a': 0}, {'a': 1}], probs=[.9, .1])
pB = Pr([{'b': 0}, {'b': 1}], probs=[.5, .5])
p = pA & pB
toDF(p)

,a,b,prob,logit
0,0,0,0.45,-0.798508
1,0,1,0.45,-0.798508
2,1,0,0.05,-2.995732
3,1,1,0.05,-2.995732


In [28]:
# Conjunction
pA1 = Pr([{'a': 0}, {'a': 1}, {'a': 2}], probs=[.7, .2, .1])
pA2 = Pr([{'a': 1}, {'a': 2}, {'a': 3}], probs=[.5, .4, .1])
p = pA1 & pA2
toDF(p.normalize())

,a,prob,logit
0,1,0.714286,-0.336472
1,2,0.285714,-1.252763


In [15]:
# Conjunction (dependent variables)
pAB = Pr([{'a': 0, 'b': 0}, {'a': 1, 'b': 1}], probs=[.9, .1])
pB = Pr([{'b': 0}, {'b': 1}], probs=[1/3, 2/3])
p = pAB & pB
toDF(p.normalize())

,a,b,prob,logit
0,0,0,0.818182,-0.200671
1,1,1,0.181818,-1.704748


In [16]:
# Conjunction (dependent variables)
pAB = Pr([{'a': 0, 'b': 0}, {'a': 1, 'b': 1}], probs=[.9, .1])
pBC = Pr([{'b': 0, 'c': 0}, {'b': 1, 'c': 0}, {'b': 1, 'c': 1}], probs=[1/3, 1/3, 1/3])
p = pAB & pBC
toDF(p.normalize())

,a,b,c,prob,logit
0,0,0,0,0.818182,-0.200671
1,1,1,0,0.090909,-2.397895
2,1,1,1,0.090909,-2.397895


# Disjunction / Mixtures

The disjunction of two distributions $p_0(A, B)$ and $p_1(B, C)$ is $p_2(A, B, C)$ where:
$$
p_2(A, B, C) \propto \alpha_0 p_0(A, B) + \alpha_1 p_1(B, C)
$$
where $\alpha_0$ and $\alpha_1$ are mixing weights.

In [26]:
# Mixture of Distributions
pA1 = Pr([{'a': 0}, {'a': 1}], probs=[.9, .1])
pA2 = Pr([{'a': 1}, {'a': 2}], probs=[.5, .5])
p = pA1*.1 | pA2*.9
toDF(p.normalize())

,a,prob,logit
0,1,0.008264,-4.795791
1,0,0.165289,-1.800058
2,2,0.826446,-0.190620


In [38]:
# Conditional Probability Example

# p(A | B = 1)
pAB1 = Pr([{'a': 0}, {'a': 1}], probs=[.9, .1])
pAB1 = assign(pAB1, "b", 1) #sets b=1 to all elements

# p(A | B = 2)
pAB2 = Pr([{'a': 0}, {'a': 1}], probs=[.2, .8])
pAB2 = assign(pAB2, "b", 2) #sets b=2 to all elements

# p(B)
pB = Pr([{'b': 1}, {'b': 2}], probs=[.5, .5])

# p(A, B) = p(A | B = 1)p(B = 1) + p(A | B = 2)p(B = 2)
pAmidB = (pAB1 | pAB2)
pAB = pAmidB & pB
toDF(pAB.normalize())

,b,a,prob,logit
0,1,0,0.45,-0.798508
1,2,0,0.10,-2.302585
2,2,1,0.40,-0.916291
3,1,1,0.05,-2.995732
